#### Customer segmentation - assignment of segment to the new customers

In [52]:
import os
import psycopg2
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
import pickle
import warnings
warnings.filterwarnings("ignore")

In [53]:
def load_data_model(from_database=False, path_to_file_data=None, path_to_file_scaler_mapper = None,  path_to_file_model = None, database_credentials = None, sql_query=None, sql_query_centroids = None):
    '''Loads the data, model and scaler mapper; data are split to new customers and already segmented customers'''
    if from_database:
        conn = psycopg2.connect(host = database_credentials[host].item(),
                                database = database_credentials[database].item(), 
                                user = database_credentials[user].item(), 
                                password = database_credentials[password].item())

        # creates a new cursor used to execute SELECT statements
        cur = conn.cursor()
        # creating the query 
        postgreSQL_select_Query = sql_query
        # quering the data
        cur.execute(postgreSQL_select_Query)
        data = cur.fetchall() 
        # puting data into a dataframe
        data = pd.DataFrame.from_records(data, columns = ['customer_id', 'gender', 'age', 'annual_income_thousands', 'spending_score'])
        
        new_data = data[data['segment'].isnull()].copy()
        segmented_data = data[data['segment'].notnull()].copy()
        
        # Loading the model
        postgreSQL_select_Query = """
                          select model_file
                          from models
                          where model = 'customer_segmentation'
                        """
  
        cur.execute(postgreSQL_select_Query)
        data = cur.fetchone() 
        model = pickle.loads(data[0])
        
        # Loading the scaler
        postgreSQL_select_Query = """
                          select scaler_file
                          from models
                          where model = 'customer_segmentation'
                        """
        # quering the data
        cur.execute(postgreSQL_select_Query)
        data = cur.fetchone() 
        scaler_mapper = pickle.loads(data[0])       
        
        
        cur.close()
        conn.close()
        
    else:
        data = pd.read_csv(path_to_file_data)
        model = pickle.load(open(path_to_file_model, 'rb'))
        scaler_mapper = pickle.load(open(path_to_file_scaler_mapper, 'rb'))
        
    new_data = data[data['segment'].isnull()].copy()
    segmented_data = data[data['segment'].notnull()].copy()
    
    return new_data, segmented_data, model, scaler_mapper

def preprocess_data(data, dummy_cols, num_cols, scaler_mapper):
    '''Creates dummy columns for categorical data and scales numerical columns'''   
    scaled_cols = []
    for col in num_cols:
        column = str(col + '_scaled')
        scaled_cols.append(column)
        data[column] = data[col]
    df = data[dummy_cols + scaled_cols]
    scaled_features = scaler_mapper.transform(df.copy())
    data[dummy_cols + scaled_cols] = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)
    display(data.head())
    return new_data, scaled_cols 

        
def dummy_encode(df, cat_cols):
    '''add to dataframe add to the dummy columns for each categorical variable and returns array with their names'''
    dummy_cols = []
    for col in cat_cols:
        df_col_dummy = pd.get_dummies(df[col])
        dummy_cols.extend(df_col_dummy.columns)
        df = pd.concat([df, df_col_dummy], axis = 1, sort = False)
    return df, dummy_cols

       
def assign_segments(model, data, dummy_cols, scaled_cols):
    '''Combine dataset with assigned segments and column labeling them as the cases were an already existed segment were assigned'''
    try:
        new_data = data[dummy_cols + scaled_cols].copy()
        data['segment'] = model.predict(new_data)
        data['segment_origin'] = 'assigned'
        data.drop(dummy_cols + scaled_cols, axis = 1, inplace = True)
        display(data.head())
        return data
  
    except:
        print('Oops, something went wrong. Check the structure of the input data and adjust the code accordingly.')

def save_data(from_database=False, path_to_file_data=None, database_credentials=None, data=None, segmented_data=None):
    '''Saves both already segmented customers and new customers with assigned segments as a one table again'''
    data = pd.concat([segmented_data, data])
    if from_database:
        engine = create_engine(engine_link = database_credentials[engine].item())  
        data.to_sql('customers', engine, if_exists = 'replace', index = False, method = 'multi')    
        engine.dispose()
    else:
        data.to_csv(path_to_file_data, index = False)
        
    return print('Segments were assigned and saved')

        
    

#### Define parameters to load saved model, scaler and new customers

In [54]:
# path to the dataset
path_to_file_data = 'Mall_Customers-New.csv'

# path to the model
path_to_file_model = 'model.pkl'

# path to the mapper of the scaler
path_to_file_scaler_mapper = 'scaler_mapper.pkl'

# Boolean value, whether dataset should be pulled from the database (PostgreSQL)
from_database = False

## Credentials to create connection to the database with psycopg2 and sqlalchemy
#database_credentials = pd.DataFrame({
#    'host': [os.environ['HOST_WAREHOUSE']],
#    'database': [os.environ['NAME_WAREHOUSE']],
#    'user': [os.environ['USER_WAREHOUSE']],
#    'password': [os.environ['PASSWORD_WAREHOUSE']],
#    'engine': [os.environ['ENGINE_WAREHOUSE']]
#})

## SQL query to download customers table
## Make sure to download the whole table as the original one will get replaced, not only specified features will be used
#sql_query =  """
#               SELECT *
#               FROM customers
#             """     


# categorical variables
cat_features = ['gender']

# Names of numerical features
numerical_features = ['age', 'annual_income_thousands', 'spending_score']


#### Load data, model and feature scaler

In [55]:
data, segmented_data, model, scaler_mapper = load_data_model(from_database=False, path_to_file_data=path_to_file_data, 
                                                             path_to_file_scaler_mapper = path_to_file_scaler_mapper, 
                                                             path_to_file_model = path_to_file_model)

#### Check whether there are new customers

In [43]:
if len(data) == 0:
    raise SystemExit("There are no new customers to assign segment to!")

#### Preprocess data

In [44]:
# encoding of new_data
new_data, cat_dummy = dummy_encode(data, cat_cols = cat_features)
print('Data with extra dummy columns:')
new_data.head()

Data with extra dummy columns:


,customer_id,gender,age,annual_income_thousands,spending_score,segment,segment_origin,Female,Male
4,5,Female,31,17,40,NaN,NaN,1,0
5,6,Female,22,17,76,NaN,NaN,1,0
6,7,Female,35,18,6,NaN,NaN,1,0
7,8,Female,23,18,94,NaN,NaN,1,0
8,9,Male,64,19,3,NaN,NaN,0,1


In [45]:
print('Data with scaled features:')
new_data, numerical_scaled = preprocess_data(new_data, dummy_cols = cat_dummy, num_cols = numerical_features, scaler_mapper = scaler_mapper)

Data with scaled features:


,customer_id,gender,age,annual_income_thousands,spending_score,segment,segment_origin,Female,Male,age_scaled,annual_income_thousands_scaled,spending_score_scaled
4,5,Female,31,17,40,NaN,NaN,0.886405,-0.886405,-0.563369,-1.662660,-0.395980
5,6,Female,22,17,76,NaN,NaN,0.886405,-0.886405,-1.209269,-1.662660,1.001596
6,7,Female,35,18,6,NaN,NaN,0.886405,-0.886405,-0.276302,-1.624491,-1.715913
7,8,Female,23,18,94,NaN,NaN,0.886405,-0.886405,-1.137502,-1.624491,1.700384
8,9,Male,64,19,3,NaN,NaN,-1.128152,1.128152,1.804932,-1.586321,-1.832378


#### Assign segments to the data

In [46]:
print('Data with assigned segments and segment origin:')
new_data = assign_segments(model = model, data = new_data, dummy_cols = cat_dummy, scaled_cols = numerical_scaled)

Data with assigned segments and segment origin:


,customer_id,gender,age,annual_income_thousands,spending_score,segment,segment_origin
4,5,Female,31,17,40,2,assigned
5,6,Female,22,17,76,2,assigned
6,7,Female,35,18,6,1,assigned
7,8,Female,23,18,94,2,assigned
8,9,Male,64,19,3,0,assigned


### Save data

In [47]:
save_data(from_database = from_database, path_to_file_data = path_to_file_data, data = new_data, segmented_data = segmented_data)

Segments were assigned and saved
